In [113]:
import json
import pandas as pd
import numpy as np

## Ribonanza

In [130]:
def rename_families(x):
    if x == '5s' or x == '16s' or x == '23s' or 'rRNA' in x:
        return 'rRNA'
    if x == 'grp1' or x == 'group_I_intron':
        return 'Introns'
    if x == 'grp2' or 'intron' in x:
        return 'Introns'
    if x == 'srp' or x == 'SRP':
        return 'SRP'
    if x == 'telomerase':
        return 'telomerase'
    if x == 'RNaseP':
        return 'rRNA'
    else: 
        return x

## RNAStralign

In [131]:
data = json.load(open('../RNAStralign/data.json'))
families = pd.DataFrame.from_dict(data, orient='index')['family'].apply(lambda x: x.split('__')[0].replace('_database', '')).apply(rename_families)
fam_rnastralign = families.value_counts()
len_rnastralign = pd.DataFrame.from_dict(data, orient='index')['sequence'].apply(len)

## archiveII

In [132]:
data = json.load(open('../archiveII/data/archiveII/data.json'))
families = pd.DataFrame.from_dict(data, orient='index').reset_index()['index'].apply(lambda x: x.split('_')[0]).apply(rename_families)
fam_archivII = families.value_counts()
len_archivII = pd.DataFrame.from_dict(data, orient='index')['sequence'].apply(len)
fam_archivII

index
rRNA          1723
SRP            725
tRNA           493
tmRNA          404
Introns        104
telomerase      37
Name: count, dtype: int64

## bpRNA (own analysis)


In [133]:
data = json.load(open('../bpRNA/data/bpRNA/data.json'))
df = pd.DataFrame.from_dict(data, orient='index').reset_index().rename(columns={'index': 'reference'})
df['family'] = df['reference'].apply(lambda x: x.split('_')[1])
df['sequence'] = df['sequence'].apply(lambda x: x.replace('T', 'U'))
df.set_index('reference', inplace=True)
print(len(df))  

66715


### bpRNA/labeled

In [134]:
valid_labels = list(fam_archivII.keys()) + list(fam_rnastralign.keys()) + ['SPR', 'RNP', 'ncRNA']
df['family'] = df['family'].apply(lambda x: 'ncRNA' if x == 'PDB' else x)
df_labeled = df[df['family'].isin(valid_labels)].copy()
df_labeled['family'].value_counts()

family
SRP      723
ncRNA    533
SPR      483
RNP      419
tmRNA    404
Name: count, dtype: int64

### bpRNA/unlabeled

In [135]:
df_un = df[~df['family'].isin(valid_labels)].copy()
df_un = df_un[['sequence']].reset_index().rename(columns={'index': 'reference'})
len(df_un)

64153

### Add labels from external databases through sequence matching

#### Rfam fasta files

In [136]:
rfam = pd.read_csv('/Users/yvesmartin/src/supermodels-data/rfam/data/rfam.csv')
rfam = rfam[['ref_desc', 'sequence', 'family_name', 'full_family_name', 'clan_name']]
rfam['sequence'] = rfam['sequence'].apply(lambda x: x.replace('T', 'U'))

In [137]:
df = pd.merge(df_un, rfam, on='sequence', how='left')
df.sort_values(by='family_name', inplace=True, na_position='last')
df.drop_duplicates(subset=['reference'], inplace=True, keep='first')
df

,reference,sequence,ref_desc,family_name,full_family_name,clan_name
182156,bpRNA_RFAM_23070,CCUUCAUUGGUUUACCUCAAACCUGUUGUGAUGUAAGUUAAUGAAG...,"Streptococcus sanguinis SK36, complete genome.",23S-methyl,23S methyl RNA motif,NaN
184249,bpRNA_RFAM_23072,CGUUUGGCGGUCGAUAUCAGCGUUUAACUGUUAGCGGCAGACAAGU...,"Lactobacillus brevis ATCC 367, complete genome.",23S-methyl,23S methyl RNA motif,NaN
126285,bpRNA_RFAM_23065,CGUUUGGUAGUUAACAUCGACAUGUCGUUGGUGACUACCGAGUUGU...,Lactobacillus plantarum WCFS1 complete genome,23S-methyl,23S methyl RNA motif,NaN
34815,bpRNA_RFAM_23059,UUUUCAUUGGUUUUUAUCAGGUUCCUGUUCUGAUAAAAGUUAGUGA...,Enterococcus villorum ATCC 700913 genomic scaf...,23S-methyl,23S methyl RNA motif,NaN
206677,bpRNA_RFAM_23077,UUUUCCCUAACUUUUAUCAGAAUACUUUUUGAUAAAAGCUAGUGAU...,Lactobacillus sakei strain 23K complete genome.,23S-methyl,23S methyl RNA motif,NaN
...,...,...,...,...,...,...
223807,bpRNA_CRW_54700,AAAGACUCAGUCCUAACCUUACUAUUGGUUUUUGCUAGACAUAUAC...,NaN,NaN,NaN,NaN
223808,bpRNA_CRW_43706,GGGUCUGUAGCUCAGUCGGUUAGAGCAGGGGACUCAUAAUCCCUUG...,NaN,NaN,NaN,NaN
223809,bpRNA_CRW_15161,GAUCCUGGCUCAGGACGAACGCUGGCGGCGUGCCUAAUACAUGCAA...,NaN,NaN,NaN,NaN
223830,bpRNA_RFAM_23681,CCCAAAGGUUCCCUCAGGCUGAAUGGAAACCAGCCAGAGAGUGUAA...,NaN,NaN,NaN,NaN


#### CRW 

In [138]:
data = pd.DataFrame.from_dict(json.load(open('../CRW/crw.json')), orient='index').reset_index().rename(columns={'index': 'reference', 'family':'family_name'})[['sequence', 'family_name']]
df = pd.merge(df, data, on='sequence', how='left')
df['family_name'] = df.apply(lambda x: x['family_name_x'] if pd.isnull(x['family_name_y']) else x['family_name_y'], axis=1)
df.drop(['family_name_x', 'family_name_y'], axis=1, inplace=True)
df['family_name'].value_counts()

family_name
tRNA              3620
5S_rRNA            848
RNaseP_bact_a      450
SAM                433
tmRNA              413
                  ... 
CbSR2                1
CbSR14               1
SBWMV2_UPD-PKl       1
CbSR1                1
TB10Cs2H2            1
Name: count, Length: 1604, dtype: int64

In [139]:
translation_family_name = {
    'sRNA': ['sRNA', 'rli', 'rivX', 'CC', 'STnc', 'sau-', 'Atu_', 'GlsR', '6C', 'ArcZ', 'Bsr', 'C0', 'CyaR_RyeE', 'whalefall-1', 'tpke11', 'tfoR',\
        't44', 'sro', 'FsrA', 'sraA', 'ryfA', 'GadY', 'GcvB', 'Hgc', 'IS009', 'InvR', 'MtlS', 'OmrA-B', 'OrzO-P', 'OxyS', 'PrrF', 'Qrr', 'RybB', 'RydC', 'SgrS',\
        'Spot_42', 'Sra'
            ],
    'tRNA': ['tRNA', 'TLS-PK'],
    'rRNA': ['rRNA', '5s', '5S_rRNA', '16s', '16S_rRNA', '23s', '23S_rRNA', 'RF_site', 'ribozyme', 'GOLLD'],
    'ncRNA': ['ncRNA', 'rdlD', 'NRON', 'CopA', 'DicF', 'uc_338', 'Dicty_Class_I_RNA', 'DsrA', 'FourU', 'srg1', 'sok', 'symR', 'sar', 'rncO', \
            'rydB', 'msr', 'IS102', 'IS128', 'MicC', 'MicF', 'NrrF', 'NsiR1', 'Plasmid_RNAIII','RNA-OUT', 'RNAI','RUF', \
             'RprA', 'Rsa', 'SprD'
                ],
    'group_I_intron': ['grp1'],
    'group_II_intron': ['grp2', 'group-II'],
    'SRP': ['srp', 'SRP'],
    'RNaseP': ['RNaseP'],
    'crRNA': ['CRISPR'],
    'RNP': ['HACA'],
    'snRNA': ['snRNA', 'sn', 'Gl_U', 'SNORA', 'U1', 'U7', 'U3', 'VA', 'SCARNA'],
    'microRNA': ['mir-', 'MIR', 'lsy-6'],
    'virus': ['virus', 'CuYV_BPYV', 'SPCSV', 'HAV', 'BMV3_UPD', 'Rubella_3', 'HIV', 'HBV', 'IRES'],
    'tmRNA': ['tmRNA'],
    'mRNA': ['SAM', 'mini-ykkC', 'FIE3'],
    'CRE': ['sucA', 'Antizyme_FSE', 'CAESAR', 'ylbH', 'yjdF', 'ydaO-yuaA', 'ybhL', 'wcaG', 'G-CSF_SLDE', 'GABA3', 'GAIT', 'speF', 'GP_knot', 
            'Gurken', 'K_chan_RES', 'Mg_sensor', 'PyrR', 'RtT', 'SECIS_'                                   ],
    'other': ['IMES-3', 'isrK', 'isrL', 'DapZ', 'ppoRNA', 'DNA', 'RyhB'],
    'telomerase': ['tp2'],
    'motif': ['MS2', 'OLE', 'PYLIS_'],
}

translation_clan_name = {
    'snRNA': ['7SK'],
    'ncRNA': ['Csr_Rsm_clan', 'FinP-traJ','Glm', 'suhB'],
    'sRNA': ['RyeA-RyeB', 'LhrC'],
    'RNaseP': ['RNaseP'],
    'mRNA': ['SL'],
}

translation_family_description = {
    'mRNA': ['riboswitch', 'UTR', 'mRNA', 'promoter', ],
    'snRNA': ['spliceosomal', 'Small nucleolar', 'snoRNA', 'Small Nucleolar RNA', 'small nucleolar'],
    'sRNA': ['sRNA', 'small RNA', 'AniS', 'Anti-Q RNA', 'Hfq binding', 'antisense','anti-sense', 'Antisense', 'anti-toxin', 'Short', 'antitoxin'],
    'virus': ['virus'],
    'microRNA': ['microRNA'],
    'tRNA': ['tRNA'],
    'tmRNA': ['tmRNA'],
    'crRNA': ['CRISPR'],
    'telomerase': ['telomerase'],
    '23S_rRNA': ['23S'],
    '6S_rRNA': ['6S'],
    'motif': ['pseudoknot', 'Hammerhead', 'motif', 'Stem loopII regulatory element in POLB', 'Pseudoknot'],
    'CRE': ['Alpha operon ribosome binding site', 'leader', 'cis-regulatory element', 'element'],
    'rRNA': ['rRNA', 'Ribosomal', 'ribosomal', 'ribozyme', 'pRNA', 'ribosome'],
    'ncRNA': ['Y RNA', 'thermometer', 'noncoding', 'SscA', 'Non-coding'], 
    'group_I_intron': ['Group I'],
}

def clean_family(row):
    x = row['clan_name']
    if not pd.isna(x):
        for family, v in translation_clan_name.items():
            for vv in v:
                if vv in x:
                    return family

    x = row['family_name']
    if pd.isna(x):
        return 'other'
    for family, v in translation_family_name.items():
        for vv in v:
            if vv in x:
                return family
            
    full_family_name = row['full_family_name']
    if pd.isna(full_family_name):
        return 'other'
    for family, keywords in translation_family_description.items():
        for kw in keywords:
            if kw in full_family_name:
                return family     
    return 'other'

def rename_families_bp2(x):
    if x == '5s' or x == '16s' or x == '23s' or 'rRNA' in x:
        return 'rRNA'
    if x == 'grp1' or x == 'group_I_intron' or x == 'Introns':
        return 'Introns'
    if x == 'grp2' or 'intron' in x:
        return 'Introns'
    if x == 'srp' or x == 'SRP':
        return 'Other'
    if x == 'telomerase':
        return 'telomerase'
    if x == 'RNaseP':
        return 'rRNA'
    if x == 'snRNA' or x == 'microRNA' or x == 'CRE' or x == 'sRNA':
        return 'sRNA'
    if x in ['virus', 'motif', 'other']:
        return 'Other'
    else: 
        return x

df_temp = df.copy()
df_temp['family'] = df.apply(clean_family, axis=1)
df_temp['length'] = df_temp['sequence'].apply(len)
df_temp.set_index('reference', inplace=True)
df_final = pd.concat([df_labeled, df_temp])[['sequence', 'family']]
df_final['family'] = df_final['family'].apply(rename_families_bp2)
bprna = df_final.value_counts('family')
len_bprna = df_final['sequence'].apply(len).tolist()

In [124]:
# label bprna 90
df90 =  open('/Users/yvesmartin/data/bpRNA/bpRNA_1m_90.fasta').read().split('\n')
refs, seqs = df90[::2], df90[1::2]
df90 = pd.DataFrame({'reference': [r.split('>')[1] for r in refs if '>' in r], 'sequence': seqs})
df90 = pd.merge(df_final, df90, on='sequence', how='inner')
bprna90 = df90.value_counts('family')
len_bprna90 = df90['sequence'].apply(len).tolist()

# Aggregate

In [125]:
df = pd.concat([bprna, bprna90, fam_rnastralign, fam_archivII], axis=1)
df.columns = ['bpRNA-1m', 'bpRNA-1m(90)', 'RNAStralign', 'ArchiveII']
# df = df.fillna(0).astype(int)
df_family = df.copy()
df_family.loc['total'] = {
    'bpRNA-1m': len(len_bprna),
    'bpRNA-1m(90)': len(len_bprna90),
    'RNAStralign': len(len_rnastralign),
    'ArchiveII': len(len_archivII),
}
# others = total - df_family.sum()

assert df_family.loc[df_family.index[:-1]].sum().sum() == df_family.loc['total'].sum()
df_family.drop('total', inplace=True)
# df_family = df_family.fillna(0).astype(int)
df_family

,bpRNA-1m,bpRNA-1m(90),RNAStralign,ArchiveII
Other,39214.0,11900.0,NaN,NaN
sRNA,15216.0,9391.0,NaN,NaN
tRNA,3828.0,1185.0,6436.0,493.0
mRNA,2470.0,1766.0,NaN,NaN
rRNA,2272.0,1289.0,18939.0,1723.0
Introns,1488.0,974.0,511.0,104.0
ncRNA,1403.0,773.0,NaN,NaN
tmRNA,971.0,520.0,170.0,404.0
SRP,NaN,NaN,193.0,725.0
telomerase,NaN,NaN,37.0,37.0


In [126]:
df_family.loc[df_family.index[:-1]].sum()

bpRNA-1m        66862.0
bpRNA-1m(90)    27798.0
RNAStralign     26249.0
ArchiveII        3449.0
dtype: float64

In [127]:
# one value per bin
def make_histograms(l):
    min_val = 0
    max_val = 4400
    bin_size = 100
    bins = np.arange(min_val + bin_size/2, max_val - bin_size/2, bin_size)
    hist, bins = np.histogram(l, bins=max_val//bin_size, range=(min_val, max_val))
    return hist, bins
hists = {
    'bpRNA-1m': make_histograms(len_bprna),
    'bpRNA-1m(90)': make_histograms(len_bprna90),
    'RNAStralign': make_histograms(len_rnastralign),
    'ArchiveII': make_histograms(len_archivII),
}

### Plot as piecharts

In [128]:
horizontal_spacing = 0.08
vertical_spacing = 0
height = 550
width = 1200

In [129]:

import plotly.graph_objects as go
from plotly.subplots import make_subplots



# first row is piechart of family distribution
# second row is histogram of sequence length distribution
# I want it to look like a paper figure
fig = make_subplots(rows=2, cols=4, specs=[[{'type': 'domain'}]*4, [{'type': 'histogram'}]*4], 
                    subplot_titles=[f"{name} (N={np.nansum(df_family[name]).astype(int)})" for name in hists.keys()],
                    vertical_spacing=vertical_spacing,
                    horizontal_spacing=horizontal_spacing,
                    row_heights=[0.5, 0.2],
)

for i in fig['layout']['annotations']:
    i['font'] = dict(size=22)
    
for i, name in enumerate(hists.keys()):
    fig.add_trace(go.Pie(
        labels=df_family.index,
        values=df_family[name],
        name=name,
        textinfo=f'percent',
        texttemplate='%{percent}',
        textposition='auto',
        showlegend=i==0,
        # sort=True,
        # legennd location
        # domain={'x': [0.0, 0.25], 'y': [0.5, 1.0]} if i == 0 else {'x': [0.25, 0.5], 'y': [0.5, 1.0]} if i == 1 else {'x': [0.5, 0.75], 'y': [0.5, 1.0]} if i == 2 else {'x': [0.75, 1.0], 'y': [0.5, 1.0]},
    ), row=1, col=i+1)
    fig.add_trace(go.Bar(
        x=hists[name][1][:-1],
        y=hists[name][0],
        name=name,
        showlegend=False,
        marker_color='rgb(0, 0, 0)',
    ), row=2, col=i+1)
    # fig.update_yaxes(row=2, col=i+1,  range=[0, 50000])
    fig.update_xaxes(row=2, col=i+1, range=[-100, 2000], tick0=0, dtick=2000)
    # make bar width constant
    # add horitontal lines
    fig.update_layout(barmode='overlay', bargap=0.1, bargroupgap=0.1)

# write 'sequence length' at the bottom, centered
fig.add_annotation(dict(
    x=0.5,
    y=-0.15,
    text='Sequence length (bin size: 100)',
    showarrow=False,
    font=dict(size=22),
    xref='paper',
    yref='paper',
    xanchor='center',
    yanchor='bottom',
))

fig.add_annotation(dict(
    x=-0.06,
    y=0.03,
    text='Count',
    showarrow=False,
    font=dict(size=22),
    xref='paper',
    yref='paper',
    xanchor='center',
    yanchor='bottom',
    textangle=-90
), 
)

fig.update_layout(
    height=height,
    width=width,
    title_x=0.5,
    title_y=0.95,
    font_size=20,
    font_family='Arial',
    font_color='black',
    legend_font_size=20,
    legend_font_family='Arial',
    legend_font_color='black',
    legend_x=1.03,
    legend_y=0.95,
    # legend_orientation='h',
    legend_traceorder='normal',
    legend_bordercolor='black',
    template="plotly_white",
)




